In [1]:
from keras.models import Model
from keras.layers import Dense,Flatten
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing import image
from keras.models import Sequential
from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
import tensorflow as tf
import os
import tensorboard

Using TensorFlow backend.


In [2]:
cd sanyam_project/

/home/saksham/Documents/sanyam_project


In [3]:
input_tensor = Input(shape=(224,224,3))
model = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
model_ft = Sequential()

for layer in model.layers:
    #layer.trainable=False
    model_ft.add(layer)

In [6]:
model_ft.add(Flatten())
model_ft.add(Dense(512, activation='relu'))
model_ft.add(Dropout(0.5))
model_ft.add(Dense(512, activation='relu'))
model_ft.add(Dropout(0.5))
model_ft.add(Dense(5, activation='softmax'))

In [7]:


import keras

sgd_apna =keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0001, amsgrad=False)
model_ft.compile(optimizer=sgd_apna, loss='categorical_crossentropy', metrics=['accuracy'])



In [8]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
batch_size = 8

#train_datagen = ImageDataGenerator()
train_datagen = ImageDataGenerator(
        #rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(224, 224),  
        batch_size=batch_size,
        class_mode='categorical'
)

Found 1010 images belonging to 5 classes.


In [9]:
pwd

'/home/saksham/Documents/sanyam_project'

In [10]:
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [11]:
validation_generator = val_datagen.flow_from_directory(
        'data/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
checkpointer = ModelCheckpoint(filepath='weights_latest_8.hdf5', verbose=1, save_best_only=True)
red=ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, mode='auto', min_delta=0.001, cooldown=0, min_lr=0)

Found 316 images belonging to 5 classes.


In [12]:
callbacks = [keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=0, batch_size=1, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)]

history = model_ft.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size ,
      epochs=400,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,callbacks=[red,checkpointer]
      )
 
# Save the model
model_ft.save('final_model.h5')

Epoch 1/400
126/126 [==============================] - 439s 3s/step - loss: 1.6805 - acc: 0.3105 - val_loss: 1.4309 - val_acc: 0.2943

Epoch 00001: val_loss improved from inf to 1.43095, saving model to weights_latest_8.hdf5
Epoch 2/400
126/126 [==============================] - 418s 3s/step - loss: 1.1400 - acc: 0.5328 - val_loss: 1.1693 - val_acc: 0.4905

Epoch 00002: val_loss improved from 1.43095 to 1.16925, saving model to weights_latest_8.hdf5
Epoch 3/400
126/126 [==============================] - 418s 3s/step - loss: 0.8167 - acc: 0.6865 - val_loss: 2.4714 - val_acc: 0.5791

Epoch 00003: val_loss did not improve from 1.16925
Epoch 4/400
126/126 [==============================] - 417s 3s/step - loss: 0.4538 - acc: 0.8561 - val_loss: 0.4748 - val_acc: 0.7500

Epoch 00004: val_loss improved from 1.16925 to 0.47478, saving model to weights_latest_8.hdf5
Epoch 5/400
126/126 [==============================] - 421s 3s/step - loss: 0.4857 - acc: 0.8423 - val_loss: 0.8280 - val_acc: 0.66

KeyboardInterrupt: 